# Trader Performance vs Bitcoin Market Sentiment

# Objective

Analyze how Bitcoin market sentiment (Fear vs Greed) impacts trader
behavior and performance on Hyperliquid, and identify trader
characteristics that remain robust across market regimes.

------------------------------------------------------------------------

# Tech Stack

-   Python
-   Jupyter Notebook
-   pandas, numpy
-   matplotlib, seaborn
-   scipy
-   scikit-learn
-   GitHub

------------------------------------------------------------------------

# Load Data

``` python
import pandas as pd

sentiment = pd.read_csv("/Users/anubhavmaandey/Downloads/fear_greed_index.csv")
trades = pd.read_csv("/Users/anubhavmaandey/Downloads/historical_data.csv")

sentiment.columns = sentiment.columns.str.strip().str.lower()
trades.columns = trades.columns.str.strip().str.lower()
```

------------------------------------------------------------------------

# Clean Sentiment Data

sentiment\[“date”\] = pd.to_datetime(sentiment\[“date”\])
sentiment\[“sentiment_flag”\] =
sentiment\[“classification”\].map({“Fear”: 0, “Greed”: 1}) sentiment =
sentiment\[\[“date”, “sentiment_flag”\]\]

------------------------------------------------------------------------

# Clean Trade Data

trades\[“trade_date”\] = pd.to_datetime(
trades.filter(regex=“time\|timestamp\|created”).iloc\[:,0\],
dayfirst=True, errors=“coerce” ).dt.date

pnl_col = next(c for c in trades.columns if “pnl” in c)
trades\[“closedpnl”\] = trades\[pnl_col\]

size_col = next(c for c in trades.columns if “size” in c or “qty” in c
or “amount” in c) trades\[“size”\] = trades\[size_col\]

trades\[“is_win”\] = (trades\[“closedpnl”\] \> 0).astype(int)
trades\[“notional”\] = trades\[“execution_price”\] \* trades\[“size”\]

------------------------------------------------------------------------

# Merge Datasets

``` python
sentiment["date"] = sentiment["date"].dt.date
merged = trades.merge(sentiment, left_on="trade_date", right_on="date", how="left")
```

------------------------------------------------------------------------

## Metrics Used to Evaluate Trader Performance

``` python
merged["pnl_pct"] = merged["closedpnl"] / merged["notional"]
merged["risk_adj_pnl"] = merged["closedpnl"] / merged["leverage"]
```

------------------------------------------------------------------------

## Initial Observations on Trader Behavior

``` python
merged.groupby("sentiment_flag")["closedpnl"].mean()
merged.groupby("sentiment_flag")["is_win"].mean()
```

------------------------------------------------------------------------

## Statistical Test

``` python
from scipy.stats import mannwhitneyu

fear = merged[merged["sentiment_flag"] == 0]["closedpnl"]
greed = merged[merged["sentiment_flag"] == 1]["closedpnl"]
mannwhitneyu(fear, greed)
```

------------------------------------------------------------------------

## Predictive Model

``` python
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = merged[["sentiment_flag","leverage","notional"]]
y = merged["is_win"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)
```

------------------------------------------------------------------------

## Key Insight Metric

``` python
robustness = merged.groupby(["account","sentiment_flag"])["closedpnl"].mean().unstack()
robustness["sentiment_robustness"] = abs(robustness[1] - robustness[0])
```

------------------------------------------------------------------------

## Short Note

1.The raw Hyperliquid dataset had inconsistent column naming for
timestamps, PnL, and trade size. I standardized these programmatically
to avoid hard-coding assumptions.

2.I focused on closed trades only because unrealized PnL introduces
noise and is not comparable across traders.

## Final Insight

Sentiment does not predict price — it predicts which traders fail.
Robust traders adjust risk when sentiment shifts.

------------------------------------------------------------------------